In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")
data4 = data4.drop(data4[data4.Label == 'Label'].index)

NORM_LEN = 2000
DDOS_LEN = 2000
BRUTE_LEN = 1500
SQL_LEN = 2500
data1 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/norm.csv")
data2 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/ddos.csv")
data3 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/nb1.csv")
data1 = data1.sample(NORM_LEN)
data2 = data2.sample(DDOS_LEN)
data3 = data3.sample(BRUTE_LEN)
df = data4.sample(SQL_LEN)
df = df.loc[df['Label'] == 'Infilteration']
df2 = data4.sample(1000)
data = pd.concat([data1, data2, data3, df, df2], ignore_index=True, sort=False)
shuffle = data.sample(frac=1, random_state=101).reset_index()

<ipython-input-2-96cf5a3fa1ae>:4: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")


In [3]:
shuffle.fillna(0, inplace=True)
df = shuffle

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
test_df = df.copy()

In [6]:
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.dropna(how="all", inplace=True)

number_inf = test_df[test_df == np.inf].count()

In [7]:
def ip2int(ip):
    if ip == 0:
      ip = '0.0.0.0'
    ip_list = list(map(int, ip.split('.')))
    num = 0
    for i in ip_list:
        num *= 256
        num += i
    return num

In [8]:
def delete_first_zero(val):
    if val[0] == '0':
        return val[1:]
    return val

def time2int(tm):
    tm_split = tm.split(' ')
    day_info = tm_split[0].split('/')
    day_info = list(map(delete_first_zero, day_info))
    day_info = list(map(int, day_info))
    day_amount = day_info[0] + (day_info[1] - 1) * 30 + (day_info[2] - 1970) * 365
    
    time_info = tm_split[1].split(':')
    time_info = list(map(delete_first_zero, time_info))
    time_info = list(map(int, time_info))
    sec_amount = 60 * (time_info[0] * 60 + time_info[1]) + time_info[2]
    
    if len(tm_split) == 3 and tm_split[-1] == 'PM':
        sec_amount += 3600 * 12
    sec_amount += day_amount * 24 * 3600
    
    return sec_amount

In [9]:
y = test_df['Label']
test_df = test_df.drop(['Label'], axis=1)
test_df['Src IP'] = test_df['Src IP'].apply(ip2int)
test_df['Dst IP'] = test_df['Dst IP'].apply(ip2int)
test_df['Timestamp'] = test_df['Timestamp'].apply(time2int)

In [10]:
test_df = test_df.drop(['Flow ID'], axis=1)

In [11]:
IMPORTANT_COLS = ['Src IP',
                  'Dst IP',
                  'Timestamp',
                  'Src Port',
                  'Dst Port',
                  'Protocol','Flow Duration',
                  'Flow Byts/s',
                  'Flow Pkts/s',
                  'Pkt Len Var',
                  'Down/Up Ratio'
                 ]

In [12]:
test_df = test_df.drop([i for i in list(test_df.columns) if i not in IMPORTANT_COLS], axis=1)
test_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,0,0.0,0,21,6,1517569940,2,0.0,1000000.0,0.0,1.0
1,316396426,37434.0,2887730457,80,6,1517684949,11659248,111.156397,0.686151,81697.0,1.0
2,2887729683,51568.0,877956772,443,6,1518059637,115317869,15.912538,0.303509,22195.113492,0.0
3,0,0.0,0,21,6,1517568392,2,0.0,1000000.0,0.0,1.0
4,0,0.0,0,21,6,1517572480,6,0.0,333333.333333,0.0,1.0


In [13]:
test_df = test_df.fillna(0)

In [16]:
test_df = test_df.astype(float)

In [17]:
df_z_scaled = test_df.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()
df_z_scaled = df_z_scaled.fillna(0)
df_z_scaled.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,-0.984024,-0.791843,-0.973354,-0.336566,-0.420241,0.545500,-0.400156,0.0,0.0,-0.333577,0.274307
1,-0.774785,0.708875,1.127716,-0.332420,-0.420241,0.546628,0.015618,0.0,0.0,0.182506,0.274307
2,0.925692,1.275503,-0.334566,-0.306914,-0.420241,0.550301,3.712139,0.0,0.0,-0.193370,-1.176870
3,-0.984024,-0.791843,-0.973354,-0.336566,-0.420241,0.545485,-0.400156,0.0,0.0,-0.333577,0.274307
4,-0.984024,-0.791843,-0.973354,-0.336566,-0.420241,0.545525,-0.400156,0.0,0.0,-0.333577,0.274307


In [18]:
y = y.str.replace('ddos', '1').str.replace('Benign','0').str.replace('FTP-BruteForce','2').str.replace('Infilteration','3')
y = y.astype(np.int64)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_z_scaled, y, test_size=0.3, random_state=42)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()
parameters={'max_depth':[3,5,10,None],
            'n_estimators':[10,100,200],
            'max_features':[1,3,5,7],
            'min_samples_leaf':[1,2,3],
            'min_samples_split':[1,2,3]
}
parameters={'max_depth':[5],
            'n_estimators':[200],
            'max_features':[7],
            'min_samples_leaf':[1],
            'min_samples_split':[2]
}

gridsearch = GridSearchCV(rfc, parameters,return_train_score=True, refit=True)
gridsearch.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
720 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/u

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 10, None],
                         'max_features': [1, 3, 5, 7],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3],
                         'n_estimators': [10, 100, 200]},
             return_train_score=True)

In [21]:
gridsearch.best_params_

{'max_depth': 5,
 'max_features': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [22]:
y_pred = gridsearch.predict(X_test)

In [23]:
from sklearn.metrics import classification_report, accuracy_score
print('Results on the test set:')
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       802
           1       1.00      1.00      1.00       591
           2       1.00      1.00      1.00       474
           3       0.86      1.00      0.92       287

    accuracy                           0.98      2154
   macro avg       0.96      0.98      0.97      2154
weighted avg       0.98      0.98      0.98      2154

0.9772516248839369
